# Assignment 2: Language Modelling in Hangman

Student Name: Yvonne Tao

Student ID: 1183577

## General info

<b>Due date</b>: Thursday, 6 April 2023 1pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 8% of mark for class (with 7% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See "Using Jupyter Notebook and Python page" on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, Gensim, Keras and Pytorch. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3.8</b>; libraries/packages should be in their latest version that is compatible with python3.8.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=https://www.nltk.org/book/>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place. In regards to the use of artificial intelligence tools in the context of academic integrity, please see the university's statement <a href="https://academicintegrity.unimelb.edu.au/plagiarism-and-collusion/artificial-intelligence-tools-and-technologies">here</a>.

# Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character-level n-gram language models. Your objective is to create an automatic player which makes the fewest mistakes.

# The Hangman Game (7 marks)

**Instructions**: The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. **No implementation is needed.**

In [93]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [94]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to `True`. When submitting your solution, set to `False` so we can automatically run the whole notebook using `Run All`.

In [95]:
interactive = False

<b>For your testing:</b>

You can play the game interactively using the following command:

In [96]:
if interactive:
    hangman('whatever', human, 8, True)

### Question 1 (1.0 mark)

**Instructions**: We will use the words in NLTK's Brown corpus for training an artificial intelligence guessing algorithm, and for evaluating the quality of the algorithm.

Your first task is to compute the number of **unique word types** occurring in the Brown corpus, using `nltk.corpus.brown` and the `words` method, and select only words that are **entirely comprised of alphabetic characters**. You should also **lowercase the words**. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the  training set. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

**Task**: Collect all unique word types from the Brown corpus, and produce `training_set` and `test_set`, 2 lists that contain 2 disjointed sets of words. Both `training_set` and `test_set` should be a python `list` (as initialised in the code). `test_set` must contain exactly 1000 word types.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [97]:
import nltk
from nltk.corpus import brown
import numpy as np

nltk.download('brown')
np.random.seed(42)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []


###
# Your answer BEGINS HERE
###
from nltk.corpus import brown
from sklearn.model_selection import train_test_split

#words from brown corpus
brown_words = brown.words()

#Select words that are entirely comprised of alphabetic characters
alphabetic_words = []
for word in brown_words:
    if word.isalpha():
        alphabetic_words.append(word.lower())
unique_alphabetic_words =list(set(alphabetic_words))

#randomly shuffle
np.random.shuffle(unique_alphabetic_words)
print(len(unique_alphabetic_words))

#split them into disjoint training and testing sets
testing_size = 1000
test_set = unique_alphabetic_words[:testing_size]
training_set = unique_alphabetic_words[testing_size:]

###
# Your answer ENDS HERE
###

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

[nltk_data] Downloading package brown to /Users/momotaro/nltk_data...
[nltk_data]   Package brown is already up-to-date!


40234
Number of word types in test = 1000
Number of word types in train = 39234


In [98]:
training_set

['broccoli',
 'forsan',
 'colony',
 'nationalist',
 'consorting',
 'beautifully',
 'twelve',
 'closets',
 'vulture',
 'pantomimic',
 'rarer',
 'powerfulness',
 'rivals',
 'phosphates',
 'chiefdom',
 'attempts',
 'gag',
 'hefted',
 'irrationality',
 'embroidery',
 'opposes',
 'grain',
 'reaches',
 'mccormick',
 'promoter',
 'stansbery',
 'stamens',
 'doin',
 'everlasting',
 'romanza',
 'daniel',
 'performers',
 'sonoma',
 'lana',
 'rekindling',
 'tempest',
 'franciscan',
 'reconciles',
 'eave',
 'vacuolated',
 'scaffolding',
 'celtic',
 'inferential',
 'ziffren',
 'repairing',
 'mechanical',
 'invigoration',
 'landslide',
 'retard',
 'herpetologist',
 'preliterate',
 'masonry',
 'eagles',
 'cutouts',
 'memorabilia',
 'unfalteringly',
 'announced',
 'rca',
 'planning',
 'obelisk',
 'fearless',
 'gags',
 'bigger',
 'diversified',
 'teahouse',
 'toynbee',
 'forego',
 'revolver',
 'jolla',
 'poughkeepsie',
 'englander',
 'grunnfeu',
 'sanctified',
 'revolutions',
 'bemaddening',
 'clinic',


<b>For your testing:</b>

In [99]:
assert(len(training_set) > 35000 and len(training_set) < 45000)
assert(len(test_set) == 1000)

**Play the game**:

Let's see how good you are at this game! Try to guess a random word from the test set. It is surprisingly difficult (and addictive)! Don't forget to set `interactive = True`.

In [100]:
#play hangman using random words from test set
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

### Question 2 (1.0 mark)

**Instructions**: To set a baseline, your first AI attempt will be a trivial **random method**. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `a ... z` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses).

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test` is provided to you. 

**Task**: Complete the `random_guesser` method. It should return a random character from the English alphabets.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [101]:
def test_guesser(guesser, test):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [102]:
import string
from string import ascii_lowercase
import random
def random_guesser(mask, guessed, **kwargs):

    ###
    # Your answer BEGINS HERE
    ###
    letters = list(ascii_lowercase)
    candidates = [letter for letter in letters if letter not in list(guessed)]
    candidate = random.choice(candidates)
    return candidate
    ###
    # Your answer ENDS HERE
    ###

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser, test_set)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = pursuit
Number of mistakes made by the random guesser = 20

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.599


<b>For your testing:</b>

In [103]:
assert(result > 10 and result < 20)

### Question 3 (1.0 mark)

**Instructions:** As your first real AI, you should train a **unigram language model** over the training set. This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

**Task**: Collect the frequencies of characters and store them in `unigram_counts` (use the first answer space). Complete the `unigram_guesser` method. Note that it takes `unigram_counts` as an additional argument (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [104]:
unigram_counts = None

###
# Your answer BEGINS HERE
###
import collections

unigram_counts = collections.defaultdict(int)
for word in training_set:
    for letter in word:
        unigram_counts[letter] = unigram_counts.get(letter,0) + 1
     
###
# Your answer ENDS HERE
###

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):

    ###
    # Your answer BEGINS HERE
    ###
    sorted_unigram_letters = sorted(unigram_counts, key=unigram_counts.get, reverse=True)
    unguessed_sorted_unigram_letters = [letter for letter in sorted_unigram_letters if letter not in guessed]
    return unguessed_sorted_unigram_letters[0]
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_guesser, test_set)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.396


<b>For your testing:</b>

In [105]:
assert(result > 5 and result < 15)

### Question 4 (1.0 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different lengths tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e.,  having a **different unigram model for each length**. You will need to be a little careful at test time, to be robust to the situation that you encounter a word length that you didn't see in training. In such a case, your method should behave like the previous `unigram_guesser` in Question 3 (i.e., it guesses characters based on unigram frequencies, unconditioned by the word length).

**Task**: Collect the frequencies of characters conditioned on the word length and store them in `unigram_counts_by_length` (use the first answer space). Complete the `unigram_length_guesser` method (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [130]:
unigram_counts_by_length = None

###
# Your answer BEGINS HERE
###
import collections

unigram_counts_by_length = collections.defaultdict(dict)
for word in training_set:
    word_leng = len(word) 
    for letter in ascii_lowercase:
        if letter in word:
            unigram_counts_by_length[word_leng][letter] = unigram_counts_by_length[word_leng].get(letter,0) + word.count(letter)
        else:
            unigram_counts_by_length[word_leng][letter] = unigram_counts_by_length[word_leng].get(letter,0) 
###
###
# Your answer ENDS HERE
###

def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):
    ###
    # Your answer BEGINS HERE
    ###
    word_length = len(mask)

    #get the unigram counts for this word
    if word_length in unigram_counts_by_length.keys():
        unigram_model = unigram_counts_by_length[word_length]
    else:
        unigram_model = unigram_counts
    
    sorted_unigram_letters = sorted(unigram_model, key=unigram_model.get, reverse=True)
    unguessed_sorted_unigram_letters = [letter for letter in sorted_unigram_letters if letter not in guessed]
    return unguessed_sorted_unigram_letters[0]
    
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_length_guesser, test_set)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.413


<b>For your testing:</b>

In [107]:
assert(result > 5 and result < 15)

### Question 5 (1.0 mark)

**Instructions:** Now for the next challenge, you'll build a **bigram language model** over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

The task here is to develop a bigram language model over characters, and train it over the training words. Remember to be careful when handling the start of each word properly, e.g., by padding with a special starting symbol such as `$`. Do we also need a special ending symbol? That's for you to decide.

Your bigram guesser should apply your language model to each blank position in the secret word by using its left context character. For example, in the partial word `e _ c _ b _ _` we know the left context for the first three blanks, but have no known left context for the last blank. In the case for the last blank, you should revert to using a unigram language model (since there's no context for us to use the bigram model). You should sum up the probability distribution (over all alphabets from <i>a</i> to <i>z</i>) for the 4 blanks, and select the alphabet with the highest probability that hasn't been guessed.

**Note**:
- When backing-off to the unigram language model, you **must use the vanilla unigram language model that you have built in Q3**. Do not use the length-based unigram language model, or any fancy variant, or you will lose marks.
- You should build a **standard bigram language model**; i.e. do not do anything complicated like a bidirectional bigram language model.

**Task**: Collect frequency counts that are necessary for building a bigram language model and store them in bigram_counts; feel free to add new objects if needed (use the first answer space). Complete the `bigram_guesser` method (use the second answer space). Note that the method currently only has one additional argument (`bigram_counts`), but you are free to add additional arguments.

In [108]:
bigram_counts = None

###
# Your answer BEGINS HERE
###

# build a bigram language model over characters, add padding "$" at the beginning of each word
bigram_counts = collections.defaultdict(int)
for word in training_set:
    word_with_padding = '$' + word
    for idx in range(1,len(word_with_padding)):
        bigram = word_with_padding[idx-1] + word_with_padding[idx]
        bigram_counts[bigram] = bigram_counts.get(bigram,0) + 1

###
# Your answer ENDS HERE
###

def bigram_guesser(mask, guessed, bigram_counts=bigram_counts, unigram_counts = unigram_counts): # add extra arguments if needed
    ###
    # Your answer BEGINS HERE
    ###
    mask_with_padding = ['$'] + mask # e.g. $remaining
    possibility_distribution = [0 for _ in range(26)]
    
    for letter_idx, letter in enumerate(ascii_lowercase):
        if letter in guessed:
            continue

        accumulated_probability = 0
        for mask_idx in range(1,len(mask_with_padding)):
            
            if mask_with_padding[mask_idx] != "_":
                continue

            left_context = mask_with_padding[mask_idx-1]
            
            if left_context != "_":
                #use bigram
                matching_bigram = {bigram: count for bigram, count in bigram_counts.items() if bigram.startswith(left_context)}
                if matching_bigram:
                    accumulated_probability += (bigram_counts[left_context+letter] / sum(matching_bigram.values()))    
            else:
                # Backoff to unigram
                accumulated_probability += unigram_counts[letter]/sum(unigram_counts.values())
        possibility_distribution[letter_idx] = accumulated_probability

    best_index = possibility_distribution.index(max(possibility_distribution))
    return ascii_lowercase[best_index]
    
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(bigram_guesser, test_set)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  8.625


### Question 6 (1.5 mark)

**Instructions:** You should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, ngram models and bidirectional models (lecture 8). Have fun!

You will be marked based on the performance of your AI model, using a pre-made training and test set (created using a secret seed). Let x be the average number of mistakes in the test set, you will score:
* 1.5 mark if x < 7.6
* 1.0 mark if 7.6 <= x < 8.0
* 0.5 mark if 8.0 <= x < 8.5
* 0.0 mark if x >= 8.5

Note:
* When testing your AI model's performance, you may want to consider trying different training/test splits (using different seeds) to have a better understanding of its performance, as there will be some variance to its performance depending on the training/test split.
* Your code must run under 3 minutes on Codalab; program that runs longer than that will be terminated and you will score 0.

**Task** Complete the `my_amazing_ai_guesser` method, which implements a better language model for hangman.

In [118]:
###
# Your answer BEGINS HERE
###

# build a trigram language model over characters, add padding "$$" at the beginning of each word
trigram_counts = collections.defaultdict(int)
for word in training_set:
    word_with_padding = '$$' + word + '$$'
    for idx in range(2,len(word_with_padding)):
        trigram = word_with_padding[idx-2] + word_with_padding[idx-1] + word_with_padding[idx]
        trigram_counts[trigram] = trigram_counts.get(trigram,0) + 1

alpha = 1

def my_amazing_ai_guesser(mask, guessed):
    # print("I am guessing", mask, "with guessed words", guessed)
    mask_with_padding = ['$'] + ['$'] + mask + ['$'] + ['$'] # e.g. $$remaining
    possibility_distribution = [0 for _ in range(26)]
    
    for letter_idx, letter in enumerate(ascii_lowercase):
        if letter in guessed:
            continue

        accumulated_probability = 0
        for mask_idx in range(2,len(mask_with_padding)-2):
            
            if mask_with_padding[mask_idx] != "_":
                continue

            most_left_context = mask_with_padding[mask_idx - 2]
            left_context = mask_with_padding[mask_idx - 1]
            
            right_context = mask_with_padding[mask_idx + 1]
            most_right_context = mask_with_padding[mask_idx + 2]

            if most_left_context == "_" and left_context == "_":
                #use unigram
                accumulated_probability += (unigram_counts[letter]+ alpha)/(sum(unigram_counts.values())+ len(unigram_counts.keys()) * alpha)
            elif left_context != "_":
                if most_left_context == "_":
                    #use bigram
                    bigram_matching_result = {bigram: count for bigram, count in bigram_counts.items() if bigram.startswith(left_context)}
                    accumulated_probability += (bigram_counts[left_context + letter] + alpha ) \
                                            / (sum(bigram_matching_result.values())+ alpha * len(bigram_counts.keys()))
                else:
                    #use trigram
                    trigram_matching_result = {trigram: count for trigram, count in unigram_counts.items() if trigram.startswith(most_left_context + left_context)}
                    accumulated_probability += (trigram_counts[most_left_context + left_context + letter] + alpha ) \
                                            / (sum(trigram_matching_result.values())+ alpha * len(trigram_counts.keys()))

        
        possibility_distribution[letter_idx] = accumulated_probability
 
    best_index = possibility_distribution.index(max(possibility_distribution))
    return ascii_lowercase[best_index]
    

###
# Your answer ENDS HERE
###

result = test_guesser(my_amazing_ai_guesser, test_set)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)


Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  8.242


In [123]:
short = [word for word in training_set if len(word) <= 5]
middile = [word for word in training_set if len(word) > 5 and len(word) <= 8]
long = [word for word in training_set if len(word) > 8]

print(len(short))
print(len(middile))
print(len(long))



7142
17754
14338


In [134]:
#build a bigram model for short words
short_bigram_counts = collections.defaultdict(int)
for word in short:
    word_with_padding = '$' + word
    for idx in range(1,len(word_with_padding)):
        bigram = word_with_padding[idx-1] + word_with_padding[idx]
        short_bigram_counts[bigram] = short_bigram_counts.get(bigram,0) + 1

#build a bigram model for middle words
middle_bigram_counts = collections.defaultdict(int)
for word in middile:
    word_with_padding = '$' + word
    for idx in range(1,len(word_with_padding)):
        bigram = word_with_padding[idx-1] + word_with_padding[idx]
        middle_bigram_counts[bigram] = middle_bigram_counts.get(bigram,0) + 1

#build a bigram model for long words
long_bigram_counts = collections.defaultdict(int)
for word in long:
    word_with_padding = '$' + word
    for idx in range(1,len(word_with_padding)):
        bigram = word_with_padding[idx-1] + word_with_padding[idx]
        long_bigram_counts[bigram] = long_bigram_counts.get(bigram,0) + 1

def length_bigram_guesser(mask, guessed):
    length = len(mask)
    if length <= 5:
        bigram_counts = short_bigram_counts
    elif length > 5 and length <= 8:
        bigram_counts = middle_bigram_counts
    else:
        bigram_counts = long_bigram_counts

    mask_with_padding = ['$'] + mask
    possibility_distribution = [0 for _ in range(26)]
    
    for letter_idx, letter in enumerate(ascii_lowercase):
        if letter in guessed:
            continue

        accumulated_probability = 0
        for mask_idx in range(1,len(mask_with_padding)):
            
            if mask_with_padding[mask_idx] != "_":
                continue

            left_context = mask_with_padding[mask_idx-1]
            
            if left_context != "_":
                #use bigram
                matching_bigram = {bigram: count for bigram, count in bigram_counts.items() if bigram.startswith(left_context)}
                accumulated_probability += (bigram_counts[left_context+letter] / sum(matching_bigram.values()))    
            else:
                # Backoff to unigram
                accumulated_probability += unigram_counts[letter]/sum(unigram_counts.values())
        possibility_distribution[letter_idx] = accumulated_probability

    best_index = possibility_distribution.index(max(possibility_distribution))
    return ascii_lowercase[best_index]
result = test_guesser(length_bigram_guesser, test_set)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  8.574


### Question 7 (0.5 mark)

**Instructions:** Explain your approach and discuss your result below. Please keep your explanation to a short paragraph.

##### Your answer BEGINS HERE

1. Add Trigram based on unigram and bigram,and Use Add-one Smoothing -  mistake drop from 8.68 to 8.248 
2. Add-k smoothing

##### Your answer ENDS HERE